In [ ]:
import pandas as pd
import numpy as np
import warnings
import re
import os
warnings.filterwarnings("ignore")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000

In [ ]:
# подгрузка всех файлов без modes2017
dr = 'modes'
file_names = os.listdir(dr)

In [ ]:
# файлы в список
mode_files = []
for name in file_names[1:]:
    mode_files.append(pd.read_excel(dr+'\\'+name,skiprows=1))
    print(name)

In [ ]:
for df in mode_files:
#     for col in df:
#         print(col)
    if 603395 in df['№ партии.1']:
        print(df.shape)

In [ ]:
del mode_files[7]['Unnamed: 42']

In [ ]:
def rep_part(ls):
    for x in ls:
        x = str(x)
        x = x.replace('.0', '')
    return ls

# for mode in mode_files:
#     mode['№ партии'] = mode['№ партии.1'].apply(rep_part)
#     mode = mode[mode['№ партии'] != 'nan']
#     print(mode['№ партии']);

In [ ]:
df_part = []
for mode in mode_files:
    for x in mode['№ партии.1']:
        for ch in ['.0']:
            df_part.append(str(x).replace(ch, ''))

df_part = rep_part(df_part)

# df_part.to_excel('df_part.xlsx')

In [ ]:
len(df_part)

In [ ]:
# for x in df_part:
#     print(x)


In [ ]:
header = mode_files[0].columns

In [ ]:
# убираем заголовки
tmp = []
for mode in mode_files:
    mode.columns = range(mode.shape[1])
    tmp.append(mode)
mode_files = tmp

# убираем последние пять столбцов из всех таблиц
for mode in mode_files:
    mode.drop(mode.columns[37:43], axis=1, inplace=True)
    
mode_files[5].shape[1]

In [ ]:
for i in mode_files:
    print(i.shape)

In [ ]:
df_modes = pd.DataFrame()
# слияние всех режимов
for df in mode_files:
    df_modes = pd.concat([df_modes,df])

df_modes.shape

In [ ]:
df_modes.shape

In [ ]:
len(header)

In [ ]:
df_modes.columns

In [ ]:
df_modes.columns = header[:37]

In [ ]:
df_modes['№ партии'] = df_part

In [ ]:
df_modes = df_modes[df_modes['№ партии'] != 'nan']
df_modes.shape

In [ ]:
df_modes.reset_index(drop=True, inplace=True)

In [ ]:
df_modes.columns
full_ls = list(df_modes.columns)
full_ls = full_ls[1:]
full_ls

In [ ]:
df_modes = df_modes.dropna(subset = ['№ партии'])
df_modes.shape

In [ ]:
df_modes[df_modes['№ партии'] ==  '-[026.xls]лист1!$m$18']

In [ ]:
df_modes.head()

In [ ]:
# df_modes[df_modes['№ партии'] == '127569']

In [ ]:
# df_part[df_part['№ партии.1'] == 612387]

In [ ]:
# dict_part = df_part.to_dict('records')
# dict_part
df_copy_for_test = df_modes.dropna(subset = ['№ партии'])
df_modes = df_copy_for_test
df_copy_for_test.shape

In [ ]:
#список усредняемых столбцов
l1 = [u'1 зона фактическая температура закалка',
        u'3 зона фактическая температура закалка',
        u'1 зона фактическая температура отпуск',
        u'3 зона фактическая температура отпуск',
        u'5 зона фактическая температура отпуск']

In [ ]:
# try:
#     df_modes[l1] = df_modes[l1].astype(float)
#     df_modes = df_modes.groupby('№ партии')[l1].apply(lambda x: x.mean())
# except:
#     print('err')
#     df_modes = df_modes.drop_duplicates('№ партии', keep = 'first')
# df_modes.shape

In [ ]:
#df1 - исходная таблица без пакетов
df_modes = df_modes.drop(['№ пакета'], axis=1)

In [ ]:
df_modes.index = df_modes['№ партии']
df_modes.head()

In [ ]:
df_modes = df_modes[df_modes['плавка'] != 'nan']
df_modes.shape

In [ ]:
# df_modes.to_excel('16022018.xlsx')

In [ ]:
df_modes.reset_index(drop=True, inplace=True)

In [ ]:
df_modes[df_modes['№ партии'] ==  '-[026.xls]лист1!$m$18']

In [ ]:
def replace_all(x):
    x = str(x)
    x = x.replace(u'X', '/')
    x = x.replace(u'x', '/')
    x = x.replace(u'Х', '/')
    x = x.replace(u'х', '/')
    x = x.replace(',', '.')
    x = x.replace('\\', '/')
    x = x.replace('/-', '/0')
    x = x.replace('-/', '0/')
    x = x.replace('***', '0')
    x = x.replace('*', '0')
    x = x.replace('-', '/')
    x = x.replace('nan', '')
    return x

def convert_step0(x):
    return replace_all(x).split('/')[0]
   
def convert_step1(x):
    try:
        return replace_all(x).split('/')[1]
    except:
        return x

df_modes = df_modes[df_modes[u'шаг балок закалочная/отпускная печи, сек']!='***']
df_modes[u'шаг балок закалочная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step0)
# df_modes[u'шаг балок закалочная печь, сек'].to_excel('check.xlsx') 
df_modes[u'шаг балок отпускная печь, сек'] = df_modes[u'шаг балок закалочная/отпускная печи, сек'].apply(convert_step1)
#del  df_modes2[u'шаг балок закалочная/отпускная печи, сек']

In [ ]:
df_modes[u'D/S'].head()

In [ ]:
def split_d(x):
    x = str(x)
    x = x.replace(',', '.')
    return re.split('х|Х|x|X', x)[0]

def split_s(x):
    x = str(x)
    x = x.replace(',', '.')
    try:
        return re.split('х|Х|x|X', x)[1]
    except:
        return x

In [ ]:
def rep_ds(x):
    x = str(x)
    x = x.replace('п', '')
    x = x.replace('П', '')
    x = x.replace('nan', '')
    x = x.replace(',', '.')
    
    return x

df_modes['диаметр'] = df_modes[u'D/S'].apply(split_d)
df_modes['толщина стенки'] = df_modes[u'D/S'].apply(split_s)
df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(rep_ds)
df_modes['толщина стенки'] = df_modes['толщина стенки'].apply(lambda x: x.replace('107.95', ''))
df_modes['диаметр'] = df_modes['диаметр'].apply(rep_ds)

In [ ]:
def rep_u_sp(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u'ек', '')
    x = x.replace(u'\мс', '')
    x = x.replace(u'm/c', '')
    x = x.replace(u'М/С', '')
    x = x.replace(u'М\С', '')
    x = x.replace(u'nan', '')
    x = x.replace(u'м/с', '')
    x = x.replace(u'м\с', '')
    x = x.replace(u',', u'.')
    x = x.replace(u'м\\', '')
    x = x.replace(u'м/', '')
    x = x.replace(u'*****', u'')
    x = x.replace(u'****', u'')
    x = x.replace(u'***', u'')
    x = x.replace(u'29.03.2017', u'')
    x = x.replace(',','.')
    x = x.replace('041','0.41')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')  
    x = x.replace('м','')
    x = x.replace('021', '0.21')
    x = x.replace('1.8', '0.18')
    x = x.replace('2.0', '0.2')
    x = x.replace('012', '0.12')
    if x == '45':
        return '0.45'
    if x == '064':
        return '0.64'
    x = x.replace(u'0.7.', '0.7')
    try:
        y = x.split('-')
        x = (float(y[0])+float(y[1]))/2
    except:
        pass
    return x
    
col = 'Скорость прохождения трубы через спрейер, м/с' 
df_modes[col] = df_modes[col].apply(rep_u_sp)
# df_modes[u'Скорость прохождения трубы через спрейер, м/с'].to_excel('check_u.xlsx')

In [ ]:
def rep_s(x):
    x = str(x)
    x = x.replace(u'сек', '')
    x = x.replace(u' ', '')
    x = x.replace(u'.', '')
    return x

df_modes[u'время выдержки в закалочной ванне, сек.'] = df_modes[u'время выдержки в закалочной ванне, сек.'].apply(rep_s)
# df_modes[u'время выдержки в закалочной ванне, сек.'].to_excel('check_t.xlsx')

In [ ]:
df_modes.shape

In [ ]:
df_modes[u'№ плавки'] = df_modes[u'плавка']
l2 = [
    'Дата термообработки', u'время начала т/о',u'D/S',u'шаг балок закалочная/отпускная печи, сек', u'плавка',
      '2 зона уставка закалка', '2 зона фактическая температура закалка', '2 зона фактическая температура отпуск', 
      '4 зона фактическая температура отпуск']
df_modes = df_modes.drop(l2, axis = 1)
df_modes.shape


In [ ]:
df_modes.shape

In [ ]:
str_to_del = [
    3.75,
    4.7,
    4.18,
    4.85,
    4.78,
    4.56,
    3.71,
    4.7,
    4.63,
    4.65,
    4.52,
    1.57,
    3.87,
    3.89,
    2.5,
    1.91,
    2.98,
    4.6,
    4.71,
    2.36,
    4.39,
    4.68,
    1.82,
    2.74,
    2.81,
    3.61
    ]

for i in str_to_del:
    df_modes.drop(df_modes[df_modes['НД (гост, ту)'] == i].index, inplace=True, axis = 0)

In [ ]:
df_modes.shape

In [ ]:
df_modes.head()

In [ ]:
df_modes.drop(df_modes[df_modes['№ партии'].isnull()].index, inplace=True, axis = 0)

In [ ]:
df_modes.shape

In [ ]:
def rep_steel(x):
    x = str(x)
    x = x.replace('092ГС','09Г2С')
    x = x.replace('13 ХФА','13ХФА')
    x = x.replace('13хфа','13ХФА')
    x = x.replace('13ХФА ','13ХФА')
    x = x.replace('13хфа-3','13ХФА-3')
    x = x.replace('15Х13H2MБ','15Х13H2МБ')
    x = x.replace('15х5мфбч','15Х5МФБЧ')
    x = x.replace('18х3мфб','18Х3МФБ')
    x = x.replace('18Х3МФБ ','18Х3МФБ')
    x = x.replace('18х3ХМФБ','18Х3МФБ')
    x = x.replace('18хзмфб','18Х3МФБ')
    x = x.replace('18хмфб','18ХМФБ')
    x = x.replace('18ХМФБ ','18ХМФБ')
    x = x.replace('20А простая','20А-пр')
    x = x.replace('20А-ПР','20А-пр')
    x = x.replace('20 А','20А')    
    x = x.replace('20A','20А') 
    x = x.replace('20А ','20А')
    x = x.replace('20 АУ','20А-У')
    x = x.replace('20АУ','20А-У')
    x = x.replace('30Г2 ','30Г2')
    x = x.replace('30Г2.','30Г2')
    x = x.replace('30г2-2','30Г2-2')
    x = x.replace('30г2ф','30Г2Ф')
    x = x.replace('30г2','30Г2')
    x = x.replace('30хгма','30ХГМА')
    x = x.replace('30ХГМА1','30ХГМА-1')
    x = x.replace('30хма','30ХМА')
    x = x.replace('32г1','32Г1')
    x = x.replace('32г2','32Г2')
    x = x.replace('32х1мф-1','32Х1МФ-1')
    x = x.replace('32ха','32ХА')
    x = x.replace('37г2ф','37Г2Ф')
    x = x.replace('37г2Ф','37Г2Ф')
    x = x.replace('37Г2ф','37Г2Ф')
    x = x.replace('37Г2Ф ','37Г2Ф')
    x = x.replace('38Г2С ','38Г2С')
    x = x.replace('38г2с-4','38Г2С-4')
    x = x.replace('38Г2-С-9','38Г2С-9')
    x = x.replace('30Г2 ', '30Г2')
    return x

df_modes['марка стали'] = df_modes['марка стали'].apply(rep_steel)

In [ ]:
def rep_vtr(x):
    x = str(x)
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('-','')
    x = x.replace('50/*','')
    x = x.replace('нет','')
    x = x.replace('НЕТ','')
    return x

df_modes['1 зона по ВТР закалка'] = df_modes['1 зона по ВТР закалка'].apply(rep_vtr)
df_modes['2 зона по ВТР закалка'] = df_modes['2 зона по ВТР закалка'].apply(rep_vtr)
df_modes['3 зона по ВТР закалка'] = df_modes['3 зона по ВТР закалка'].apply(rep_vtr)


In [ ]:
def rep_nd(x):
    x = str(x)
    x = x.replace('н/д','')
    x = x.replace('.0', '')
    x = x.replace('НЕТ', '')
    x = x.replace('***', '')
    return x

df_modes['1 зона уставка закалка'] = df_modes['1 зона уставка закалка'].apply(rep_nd)
df_modes['1 зона фактическая температура закалка'] = df_modes['1 зона фактическая температура закалка'].apply(rep_nd)
df_modes['3 зона уставка закалка'] = df_modes['3 зона уставка закалка'].apply(rep_nd)
df_modes['3 зона фактическая температура закалка'] = df_modes['3 зона фактическая температура закалка'].apply(rep_nd)
df_modes['Температура трубы на выгрузке из печи закалки'] = df_modes['Температура трубы на выгрузке из печи закалки'].apply(rep_nd)
df_modes['Расход воды на закалочный спрейер, м3'] = df_modes['Расход воды на закалочный спрейер, м3'].apply(rep_nd)
df_modes['t˚ C воды в закалочной ванне'] = df_modes['t˚ C воды в закалочной ванне'].apply(rep_nd)
df_modes['1 зона фактическая температура отпуск'] = df_modes['1 зона фактическая температура отпуск'].apply(rep_nd)
df_modes['3 зона ВТР и уставка отпуск'] = df_modes['3 зона ВТР и уставка отпуск'].apply(rep_nd)
df_modes['3 зона фактическая температура отпуск'] = df_modes['3 зона фактическая температура отпуск'].apply(rep_nd)
df_modes['5 зона ВТР и уставка отпуск'] = df_modes['5 зона ВТР и уставка отпуск'].apply(rep_nd)
df_modes['5 зона фактическая температура отпуск'] = df_modes['5 зона фактическая температура отпуск'].apply(rep_nd)
df_modes['Температура трубы на выгрузке из печи отпуска'] = df_modes['Температура трубы на выгрузке из печи отпуска'].apply(rep_nd)

In [ ]:
def rep_time(x):
    x = str(x)
    x = x.replace('nan','')
    x = x.replace('НЕТ','')
    x = x.replace('нет','')
    x = x.replace('НОРМ','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('40-50','45')
    x = x.replace('сек','')
    x = x.replace('с','')
    x = x.replace(',','.')
    x = x.replace('11cек', '')
    return x


col ='время выдержки в закалочной ванне, сек.'
df_modes[col] = df_modes[col].apply(rep_time)

In [ ]:
def rep_tempr(x):
    x = str(x)
    x = x.replace('******','')
    x = x.replace('*****','')
    x = x.replace('****','')
    x = x.replace('***','')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция', '')
    x = x.replace('\\','/')
    x = x.replace('-','/')
    x = x.replace('<', '')
    x = x.replace('>', '')
    x = x.replace('Доотпуск', 'доотпуск')
    x = x.replace('до', '')
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('БизнесТренд', '')
    x = x.replace('ВОЗДУХ', '')
    x = x.replace('Воздух', '')
    x = x.replace('воэдух', '')
    x = x.replace('высадка', '')
    x = x.replace('закалка', '')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('нет', '')
    x = x.replace('НОРМ', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('воздух', '')
    x = x.replace('61702', '')
    x = x.replace('опытные.','')
    x = x.replace('на е','')
    x = x.replace('норм/ия','')
    x = x.replace(',','.')
    y = x.split('/')
    if len(y)==1:
        return x
    fl = True
    tmpy= []
    for i in y:
        if len(i) == 4:
            y.append(i[0:2])
            y.append(i[2:4])
            y.remove(i)
    for i in y:
        try:
            tmpy.append(float(i))
        except:
            tmpy.append(i)
            fl = False
    if(fl):
        y = tmpy   
        x = np.mean(y)
    if x=='0':
        x=''
    return x

col = 't˚ C трубы после спреера'
# df_modes[col].to_excel('check_col_before.xlsx')
df_modes[col] = df_modes[col].apply(rep_tempr)
df_modes.drop(df_modes[df_modes[col]=='отпуск'].index, inplace = True, axis = 0)
df_modes[col] = df_modes[col].apply(
                lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float)
# df_modes[col].to_excel('check_col.xlsx')

In [ ]:
def rep_tempr_bath(x):
    x = str(x)
    x = x.replace('Бизнес Тренд', '')
    x = x.replace('бизнес тренд', '')
    x = x.replace('высадка.', '')
    x = x.replace('высадка', '')
    x = x.replace('ВЫСАДКА', '')
    x = x.replace('Высадка', '')
    x = x.replace('высадка', '') 
    x = x.replace('доотпуск.', 'отпуск')
    x = x.replace('доотпуск', 'отпуск')
    x = x.replace('Доотпуск', 'отпуск')
    x = x.replace('отпуск.', 'отпуск')
    x = x.replace('на отпуск', 'отпуск')
    x = x.replace('в малахи', '')
    x = x.replace('ДООТПУСК', 'отпуск')
    x = x.replace('НЕТ', '')
    x = x.replace('нет', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('НК 8500', '')
    x = x.replace('НК 9000', '')
    x = x.replace('нормализ.', '')
    x = x.replace('нормализация', '')
    x = x.replace('нормал-ция', '')
    x = x.replace('НОРМ-ИЯ', '')
    x = x.replace('норм-ция.', '')
    x = x.replace('норм-ция', '')
    x = x.replace('нор-ция', '')
    x = x.replace('нор/ция', '')
    x = x.replace('опытные.', '')
    x = x.replace('ОПЫТНЫЕ', '')
    x = x.replace('повторная', '')
    x = x.replace('повтор', '')
    x = x.replace('ПОВТОР', '')
    x = x.replace('Самара', '')
    x = x.replace('СИТЦ', '')
    x = x.replace('Трэнд', '')
    x = x.replace('ЧПТЗ', '')
    x = x.replace('ЧТПЗ', '')
    x = x.replace('30 СЕК.', '30')
    x = x.replace('2/я зак/ка', '')
    x = x.replace('ТРЕНД', '')
    x = x.replace('ПЛ.Р. Ц42/№10','')
    x = x.replace('Челябинск','')
    x = x.replace('нормал/ция', '')
    if x=='0':
        x=''
    if x=='.':
        x=''
    return x
col = 't˚ C трубы после ванны'
# df_modes[col].to_excel('check_col_before.xlsx')
df_modes[col] = df_modes[col].apply(rep_tempr)
df_modes[col] = df_modes[col].apply(rep_tempr_bath)
df_modes.drop(df_modes[df_modes['t˚ C трубы после ванны']=='отпуск'].index, inplace = True, axis = 0)
df_modes[col] = df_modes[col].apply(
                lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float)
# df_modes[col].to_excel('check_col.xlsx')

In [ ]:
# df_modes[df_modes['t˚ C трубы после ванны'] == 'ПЛ.Р. Ц42/№10']

In [ ]:
def rep_vtr(x):
    x = str(x)
    x = x.replace('н/д','')
    x = x.replace('***','')
    return x


df_modes['1 зона ВТР и уставка отпуск'] = df_modes['1 зона ВТР и уставка отпуск'].apply(rep_vtr)

In [ ]:
def rep_vtr2(x):
    x = str(x)
    x = x.replace('***', '')
    x = x.replace('****', '')
    x = x.replace('НЕТ', '') 
    x = x.replace('нет', '')
    if x == '67':
        return '670'
    if x == '69':
        return '690'
    return x

df_modes['2 зона ВТР и уставка отпуск'] = df_modes['2 зона ВТР и уставка отпуск'].apply(rep_vtr2)
df_modes['4 зона ВТР и уставка отпуск'] = df_modes['4 зона ВТР и уставка отпуск'].apply(rep_vtr2)

In [ ]:
df_modes.shape

In [ ]:
#186
df_modes.drop(df_modes[df_modes['шаг балок закалочная печь, сек'] == '0'].index, inplace=True, axis=0)
#204
df_modes.drop(df_modes[df_modes['шаг балок отпускная печь, сек'] == '0'].index, inplace=True, axis=0)

In [ ]:
df_modes.shape

In [ ]:
ls_mean = df_modes.columns[4:-3]
list(ls_mean)

In [ ]:
def get_mean(df):
#     df = df.groupby(['№ плавки', '№ партии']).first()
#     print(df.head())
    for col in ls_mean:
        if col in df:
            print(col)
#             try:
            df[col] = df.groupby(['№ плавки', '№ партии'])[col].transform(
                lambda x: x.apply(
                lambda y: str(y) if (y!='nan')&(y!='')&(y!=None)&(y!=' ') else None).astype(float).mean())
#             except:
#                 print(col)
#                 pass
#             df[col] = df[col].apply(lambda x: str(x) if (x!='nan')&(x!='')&(x!=None)&(x!=' ') else None).astype(float)
            
#     df = df.reset_index()
    return df

In [ ]:
df_modes['№ партии'] = df_modes['№ партии'].astype(str).str.lower()
df_modes['№ плавки'] = df_modes['№ плавки'].astype(str).str.lower()

In [ ]:
df_modes.shape

In [ ]:
tmp = get_mean(df_modes)
# tmp.groupby('№ партии').shape
tmp.shape

In [ ]:
# tmp.to_excel('check_gr.xlsx')

In [ ]:
# tmp = pd.read_excel('check_gr.xlsx')

In [ ]:
full_ls = list(df_modes.columns)
# print(len(full_ls))
full_ls = full_ls[-1:]+full_ls[0:-1]
# print(len(full_ls))
full_ls.remove('кол-во, шт.') 
# full_ls

In [ ]:
# tmp[tmp['№ партии'].duplicated()]
tmp = tmp[full_ls]
tmp = tmp.drop_duplicates()
tmp.to_excel('for_comparision.xlsx')
tmp.shape

In [ ]:
tmp['2 зона по ВТР закалка'] = tmp['2 зона по ВТР закалка'].apply(lambda x: x*10 if x<100 else x)
tmp.shape

In [ ]:
tmp.to_excel('all_modes_clean_2.xlsx')